In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import torchvision
import torch
from torchvision import transforms
from torchvision import datasets
import torch.nn as nn
import torch.nn.functional as F

import time
from torch.utils.tensorboard import SummaryWriter
from ignite.handlers import ModelCheckpoint

from ignite.engine import Engine, Events
from ignite.metrics import Loss, Accuracy
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score


In [2]:
class HandModel(torch.nn.Module):
    def __init__(self):
        super(type(self), self).__init__()
        self.c1 = nn.Conv2d(3, 32, kernel_size=11, stride=4, padding=2)
        self.c2 = nn.Conv2d(32,96, kernel_size=5, padding=2)
        self.c3 = nn.Conv2d(96,192, kernel_size=3, padding=1)
        self.c4 = nn.Conv2d(192, 128, kernel_size=3, padding=1)
        self.c5 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.ac = nn.ReLU(inplace=True)
        self.mp = nn.MaxPool2d(kernel_size=3, stride=2)
        self.ap = nn.AdaptiveAvgPool2d((3, 3))
        self.f1 = nn.Linear(1152, 512)
        self.f2 = nn.Linear(512, 512)
        self.f3 = nn.Linear(512, 4)
    def forward(self, x):
        x = self.mp(self.ac(self.c1(x)))
        x = self.mp(self.ac(self.c2(x)))
        x = self.ac(self.c3(x))
        x = self.ac(self.c4(x))
        x = self.mp(self.ac(self.c5(x)))
        x = self.ap(x)
        x = torch.flatten(x, 1)
        x = self.ac(self.f1(x))
        x = self.ac(self.f2(x))
        x = self.f3(x)
        return x

In [ ]:
# Cargar Modelo

model = HandModel()
model.load_state_dict(torch.load('./HandModel-117.pt',map_location=torch.device('cpu')))
display(model)

In [ ]:
# Visualizamos Filtros Resultantes

w = model.c1.weight.data.numpy()
M = w.shape[0]
fig, ax = plt.subplots(1, M, figsize=(15, 2), tight_layout=True)

for i in range(M):    
    ax[i].imshow(w[i, 0, :, :])
    ax[i].axis('off')
    
w = model.c2.weight.data.numpy()
M = w.shape[0]
fig, ax = plt.subplots(1, M, figsize=(60, 4), tight_layout=True)

for i in range(M):    
    ax[i].imshow(w[i, 0, :, :])
    ax[i].axis('off')

w = model.c3.weight.data.numpy()
M = w.shape[0]
fig, ax = plt.subplots(1, M, figsize=(60, 4), tight_layout=True)

for i in range(M):    
    ax[i].imshow(w[i, 0, :, :])
    ax[i].axis('off')

w = model.c4.weight.data.numpy()
M = w.shape[0]
fig, ax = plt.subplots(1, M, figsize=(60, 4), tight_layout=True)

for i in range(M):    
    ax[i].imshow(w[i, 0, :, :])
    ax[i].axis('off')
    
w = model.c5.weight.data.numpy()
M = w.shape[0]
fig, ax = plt.subplots(1, M, figsize=(60, 4), tight_layout=True)

for i in range(M):    
    ax[i].imshow(w[i, 0, :, :])
    ax[i].axis('off')

In [ ]:
# Evaluar 1 modelo

transform = transforms.Compose(
[
    transforms.ToTensor()
])
model = HandModel()
model.load_state_dict(torch.load('./HandModel-117.pt',map_location=torch.device('cpu')))    
test_dataset = datasets.ImageFolder('../gestos/test', transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset,shuffle=False, batch_size=200)

test_targets = test_dataset.targets
prediction_test = []

for mbdata, label in test_loader:
    logits = model.forward(mbdata)
    prediction_test.append(logits.argmax(dim=1).detach().numpy())
prediction_test = np.concatenate(prediction_test)

cm = confusion_matrix(test_targets, prediction_test)
display(cm)

clas = classification_report(test_targets, prediction_test)
print(clas)
print(accuracy_score(test_targets, prediction_test))

In [ ]:
# Evaluamos todos los modelos
bestMod = 1
bestAcc = 0
for i in range(158):
    model = HandModel()
    model.load_state_dict(torch.load('./HandModel-'+str(i+1)+'.pt',map_location=torch.device('cpu')))
    test_dataset = datasets.ImageFolder('../gestos/test', transform=transform)
    test_loader = torch.utils.data.DataLoader(test_dataset,shuffle=False, batch_size=200)

    test_targets = test_dataset.targets
    prediction_test = []
    for mbdata, label in test_loader:
        logits = model.forward(mbdata)
        prediction_test.append(logits.argmax(dim=1).detach().numpy())
    prediction_test = np.concatenate(prediction_test)
    
    acu = accuracy_score(test_targets, prediction_test)
    if(acu > bestAcc):
        bestMod = i+1
        bestAcc = acu
        
print(bestMod)
print(bestAcc)

In [ ]:
# Probar imágenes propias

model = HandModel()
model.load_state_dict(torch.load('./HandModel-117.pt',map_location=torch.device('cpu')))
test_dataset = datasets.ImageFolder('../t', transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset,shuffle=False, batch_size=1)
logits = model.forward(test_dataset[0][0].unsqueeze(0))
print(logits.argmax(dim=1).detach().numpy()[0])